In [ ]:
%pip install tensorflow tensorflow-gpu opencv-python matplotlib,numpy

**Installing Indispendable dependencies**


*   Tensorflow
*   Tensorflow-gpu
*   Opencv-python
*   Matplotlib
*   Numpy


In [ ]:
import os
import numpy as np # type: ignore
import keras # type: ignore
from keras import layers # type: ignore
from tensorflow import data as tf_data # type: ignore
import matplotlib.pyplot as plt # type: ignore

Importing the install dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_folder = '/content/drive/My Drive/Dataset/PetImages'
import os
os.listdir(dataset_folder)



Connect Colab with google drive to get the dataset.

In [ ]:
corrupted_count = 0  # Counter for corrupted images

# Ensure the dataset folder exists
if not os.path.exists(dataset_folder):
    print(f"Dataset folder '{dataset_folder}' not found.")
else:
    for category in ("Cat", "Dog"):
        category_path = os.path.join(dataset_folder, category)
        if not os.path.exists(category_path):
            print(f"Category folder '{category_path}' not found. Skipping.")
            continue

        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            try:
                with open(image_path, "rb") as image_file:
                    is_valid_jpeg = b"JFIF" in image_file.peek(10)
            except Exception as e:
                # Handle unexpected errors during file access
                print(f"Error reading file {image_path}: {e}")
                is_valid_jpeg = False

            if not is_valid_jpeg:
                corrupted_count += 1
                # Delete corrupted image
                try:
                    os.remove(image_path)
                    print(f"Deleted corrupted file: {image_path}")
                except Exception as e:
                    print(f"Error deleting file {image_path}: {e}")

print(f"Deleted {corrupted_count} corrupted images.")


Clean the dataset by evicting:

*   Corrupted images
*   Damaged images
*   Not in the required format

In [ ]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    dataset_folder,
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

Visulizing the Images in the require format

In [ ]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

In [ ]:
image_size = (180, 180)
num_channels = 3
input_shape = (*image_size, num_channels)

# Model inputs and preprocessing
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)


In [ ]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

In [ ]:
img = keras.utils.load_img("/content/drive/My Drive/Datasets/PetImages/Cat/12499.jpg", target_size=image_size)
plt.imshow(img)

img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")

Running Inferencing on new data

In [ ]:
!git clone https://github.com/Mubashir714/Image-Classification.git